# Welcome to the Lab 🥼🧪

Visualize share of investor purchases relative to US pricefeed 

**Note** This notebook will work with any of the dozens of markets we have real time pricefeeds for. It will work for all 70k housing markets if scoped to investor purchase prices vs. sales activity

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/inspiration/pricefeed_vs_share_of_investor_purchases.ipynb)

In [ ]:
# Environment setup
import os
import sys
import json
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido numpy
    !git clone https://github.com/ParclLabs/parcllabs-examples.git
    sys.path.append('/content/parcllabs-examples/python/')
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')
    cur_dir = os.getcwd()
    chart_dir = os.path.join(cur_dir, '..')
    sys.path.append(chart_dir)

In [ ]:
import parcllabs
import pandas as pd
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from charting.utils import create_labs_logo_dict, format_metro_names

api_key = os.getenv('PARCL_LABS_API_KEY')
print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
cur_dir = os.getcwd()
chart_dir = os.path.join(cur_dir, '..')
sys.path.append(chart_dir)

from charting.utils import create_labs_logo_dict

In [ ]:
# Initialize the Parcl Labs client
client = ParclLabsClient(api_key)

In [ ]:
# set nb config
pf_options = {
    'rental': 'rental_price_feed',
    'pricefeed': 'price_feed'
}

PF_TYPE = pf_options['pricefeed']

In [ ]:
# lets get all US markets currently available to trade on the Parcl Exchange
# Now lets say you want all price feed markets that are on the parcl exchange
market_df = client.search_markets.retrieve(
    sort_by='PARCL_EXCHANGE_MARKET',
    sort_order='DESC',
    as_dataframe=True,
    params={'limit': 14},  # expand the default limit to 14, as of this writing, 14 markets are available
)

parcl_ids = market_df['parcl_id'].tolist()
market_df.head()

In [ ]:
# lets retrieve data back to 2011 for these price feeds
START_DATE = '2019-01-01'
feeds = client.price_feed.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
    auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)
    
investor_purchases = client.investor_metrics_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
)

In [ ]:
counts = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
)

In [ ]:
investor_prices = client.investor_metrics_housing_event_prices.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
)

In [ ]:
counts = counts.rename(columns={'sales': 'all_sales'})

usa_counts = counts.loc[counts['parcl_id'] == 5826765]
investor_usa_counts = investor_purchases.loc[investor_purchases['parcl_id'] == 5826765]
investor_usa_prices = investor_prices.loc[investor_prices['parcl_id'] == 5826765]
pf = feeds.loc[feeds['parcl_id'] == 5826765]

In [ ]:
usa_counts = usa_counts.merge(investor_usa_counts[['date', 'parcl_id', 'acquisitions']], on=['date', 'parcl_id'], how='inner')
usa_counts['pct_investor_purchases'] = usa_counts['acquisitions'] / usa_counts['all_sales']
usa_counts


In [ ]:
labs_logo_dict = create_labs_logo_dict()

In [ ]:
def build_dual_axis_chart(
        market_name: str, 
        data: pd.DataFrame,
        investor_data: pd.DataFrame,
        price_series: str = 'price_feed',
        investor_series: str = 'investor_purchases_percent',
        save_path: str=None,
        title: str=None,
        yaxis1_title: str=None,
        yaxis2_title: str=None
    ):

    HEIGHT = 900
    WIDTH = 1600
    
    fig = go.Figure()

    # Add primary y-axis trace for price series
    fig.add_trace(go.Scatter(
        x=data['date'],
        y=data[price_series],
        mode='lines',
        line=dict(width=3, color='#00BFFF'),  # White color for price series
        name=yaxis1_title,
    ))

    # Add bar trace for investor purchases
    fig.add_trace(go.Bar(
        x=investor_data['date'],
        y=investor_data[investor_series],
        marker=dict(color='gray', opacity=0.6),
        name=yaxis2_title,
        yaxis='y2'
    ))

    fig.add_layout_image(labs_logo_dict)
    
    fig.update_layout(
        margin=dict(l=0, r=0, t=110, b=0),
        height=HEIGHT,
        width=WIDTH,
        title={
            'text': title,
            'y': 0.99,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=28, color='#FFFFFF'),
        },
        plot_bgcolor='#000000',
        paper_bgcolor='#000000',
        font=dict(color='#FFFFFF'),
        xaxis=dict(
            title_text='',
            showgrid=False,
            tickangle=-45,
            tickfont=dict(size=14),
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1
        ),
        yaxis=dict(
            title_text=yaxis1_title,
            showgrid=True,
            gridwidth=0.5,
            gridcolor='rgba(255, 255, 255, 0.2)',
            tickfont=dict(size=14),
            tickprefix='$',
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1
        ),
        yaxis2=dict(
            title_text=yaxis2_title,
            showgrid=False,
            tickfont=dict(size=14),
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1,
            overlaying='y',
            side='right',
            tickformat=".0%"
        ),
        hovermode='x unified',
        hoverlabel=dict(
            bgcolor='#1F1F1F',
            font_size=14,
            font_family="Rockwell"
        ),
        legend=dict(
            x=0.95,  # Position legend in the bottom right corner
            y=0.01,
            xanchor='right',
            yanchor='bottom',
            font=dict(size=14, color='#FFFFFF'),
            bgcolor='rgba(0, 0, 0, 0.5)'
        )
    )

    if save_path:
        fig.write_image(save_path, width=WIDTH, height=HEIGHT)
    fig.show()


def build_stacked_bar_chart(
    market_name: str, 
    data: pd.DataFrame,
    sales_data: pd.DataFrame,
    price_series: str = 'price_feed',
    retail_series: str = 'retail_sales',
    acquisition_series: str = 'acquisitions',
    save_path: str = None,
    title: str = None,
    yaxis1_title: str = None,
    yaxis2_title: str = None
):

    HEIGHT = 900
    WIDTH = 1600
    
    fig = go.Figure()

    # Add bar trace for investor acquisitions first (bottom of the stack)
    fig.add_trace(go.Bar(
        x=sales_data['date'],
        y=sales_data[acquisition_series],
        marker=dict(color='gray', opacity=0.5),
        name='Investor Purchases',
        yaxis='y2'
    ))

    # Add bar trace for retail sales second (top of the stack)
    fig.add_trace(go.Bar(
        x=sales_data['date'],
        y=sales_data[retail_series],
        marker=dict(color='lightgray', opacity=0.5),
        name='Retail Purchases',
        yaxis='y2'
    ))

    # Add primary y-axis trace for price series
    fig.add_trace(go.Scatter(
        x=data['date'],
        y=data[price_series],
        mode='lines',
        line=dict(width=3, color='#00BFFF'),  # Blue color for price series
        name=yaxis1_title,
    ))

    # Add logo image
    fig.add_layout_image(labs_logo_dict)
    
    fig.update_layout(
        margin=dict(l=0, r=0, t=110, b=0),
        height=HEIGHT,
        width=WIDTH,
        title={
            'text': title,
            'y': 0.99,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=28, color='#FFFFFF'),
        },
        plot_bgcolor='#000000',
        paper_bgcolor='#000000',
        font=dict(color='#FFFFFF'),
        xaxis=dict(
            title_text='',
            showgrid=False,
            tickangle=-45,
            tickfont=dict(size=14),
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1
        ),
        yaxis=dict(
            title_text=yaxis1_title,
            showgrid=True,
            gridwidth=0.5,
            gridcolor='rgba(255, 255, 255, 0.2)',
            tickfont=dict(size=14),
            tickprefix='$',
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1
        ),
        yaxis2=dict(
            title_text=yaxis2_title,
            showgrid=False,
            tickfont=dict(size=14),
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1,
            overlaying='y',
            side='right'
        ),
        hovermode='x unified',
        hoverlabel=dict(
            bgcolor='#1F1F1F',
            font_size=14,
            font_family="Rockwell"
        ),
        legend=dict(
            x=0.95,  # Position legend in the bottom right corner
            y=0.01,
            xanchor='right',
            yanchor='bottom',
            font=dict(size=14, color='#FFFFFF'),
            bgcolor='rgba(0, 0, 0, 0.5)'
        ),
        barmode='stack'  # Stack the bars
    )

    if save_path:
        fig.write_image(save_path, width=WIDTH, height=HEIGHT)
    fig.show()


In [ ]:


# Example call to the function
build_dual_axis_chart('USA', 
                      investor_usa_prices, 
                      usa_counts,
                      price_series='price_median_acquisitions',
                      investor_series='pct_investor_purchases',
                      yaxis1_title='Median Investor Purchase Price ($)',
                      yaxis2_title='Percent Investor Purchases',
                      title='National Median Investor Purchase Price and Percent Investor Purchases',
                      )

In [ ]:


usa_counts['retail_sales'] = usa_counts['all_sales'] - usa_counts['acquisitions']

# Example call to the function
build_stacked_bar_chart(
    'USA', 
    pf, 
    usa_counts,
    price_series='price_feed',
    retail_series='retail_sales',
    acquisition_series='acquisitions',
    yaxis1_title='Price per Square Foot ($)',
    yaxis2_title='Total Number of Sales',
    title='National Price per Square Foot and Total Number of Sales'
)
